In [1]:
!pip install tenseal
!pip install numpy
!pip install matplotlib

In [2]:
import copy
import tenseal as ts
import time
import numpy as np
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools
import random

In [3]:
def max_with_lp(distances, p=20):
  #alpha = ts.plain_tensor(1/p)
  alpha = 1/p
  #return taylor_series_sqrt_x_plus_1_alpha((x1**p + x2**p+ x3**p+ x4**p) - 1, alpha)
  result_pow = [dist**p for dist in distances]
  return sum(result_pow)
  #return (x1**p + x2**p+ x3**p+ x4**p)


def taylor_series_sqrt_x_plus_1_alpha(x, a):
  sum_elements = 1
  #print("a={}, x={}".format(a,x.data))
  sum_elements += x*a
  sum_elements += 0.5*a*(a-1)*(x**2)
  sum_elements += (1/6)*a*(a-1)*(a-2)*(x**3)
  return sum_elements


def distance(a, b):
  return a-b


def get_p_value(x, max_val, p,n):
  approx = approx_to_1_div_x(max_val,n)
  xp = x**p
  #print("x = {}, max_val = {}".format(x, max_val))
  #return xp/max_val
  prob = xp * approx
  return prob
  


#converges only when 0<x<2
#was n=3
def approx_to_1_div_x(x, n):
  #print("x in approx_to_1_div_x is: {}".format(x))
  mul = 1
  for i in range(n):
    mul *= (1+(1-x)**(2**i))
  #print("1/{} = {}".format(x,mul))
  return mul

def prob_to_be_in_center(data, centers, p,n):
  num_of_centers = len(centers)
  num_of_points = len(data)
  distances = []
  sum_distance_p_from_center = []
  for x in data:
      temp_dis = []
      temp_sum = 0
      for c in centers:
            dis = distance(x, c)
            temp_dis.append(dis)
            dis_pow = dis**p
            temp_sum += dis_pow

      sum_distance_p_from_center.append(temp_sum)
      distances.append(temp_dis)
    
    
      #temp_dis = [distance(x, c) for c in centers]
      #distances.append(temp_dis)
      #sum_distance_p_from_center.append(sum([item**p for item in temp_dis]))

  probs = []
  p_values_sum = []
  for i in range(num_of_centers):
    probs_sum = 0
    p_values_sum_per_center = 0
    for j in range(num_of_points):
        p_value = get_p_value(distances[j][i], sum_distance_p_from_center[j], p,n)
        probs_sum += data[j] * (1 - p_value)
        p_values_sum_per_center += (1 - p_value)
        #print("@@@@@")
        #print("point: {}, center: {}, 1 - p_value: {}".format(data[j], centers[i], 1 - p_value)) 
    probs.append(probs_sum)
    p_values_sum.append(p_values_sum_per_center)

  return probs, p_values_sum


def get_ckks_context():
  # parameters
  #poly_mod_degree = 2**12
  poly_mod_degree = 2**13
  #coeff_mod_bit_sizes = [40, 20, 40]
  coeff_mod_bit_sizes = [35, 25, 25, 25, 25, 25, 25, 25, 25, 35]
  # create TenSEALContext
  context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  # scale of ciphertext to use
  #context.global_scale = 2 ** 20
  context.global_scale = 2 ** 25
  # this key is needed for doing dot-product operations
  context.generate_galois_keys()
  return context


def get_other_context():
  # parameters
  #poly_mod_degree = 16384
  poly_mod_degree = 32768
  #coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
  #coeff_mod_bit_sizes = [60,35,35,35,35,35,35,35,35,35,60]
  coeff_mod_bit_sizes = [60,50,50,50,50,50,50,50,50,50,50,50,60]
  print("sum of coeff_mod_bit_sizes=", sum(coeff_mod_bit_sizes))
  # create TenSEALContext
  ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  ctx_training.global_scale = 2 ** 50
  ctx_training.generate_galois_keys()
  return ctx_training


def get_bfv_context():
  context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
  return context

def print_fig(X, centers_colors, plot_name):
    # Create a scatter plot
    fig, axs = plt.subplots(nrows=1, ncols=1)
    axs.scatter(X, [0]*len(X), c=centers_colors)
    axs.set_title(plot_name)
    fig.show()

In [4]:
def our_kmeans(data, centers, n, p, iterations):
    print("our_kmeans...")
    start = time.time()
    probs, p_values_sum = prob_to_be_in_center(data, centers,p,n)
    new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
    end = time.time()

    #print("({}/{}) centers = {}, Duration: {} seconds".format(1, iterations, new_centers, end - start))

    for i in range(1, iterations):
        start = time.time()
        probs, p_values_sum = prob_to_be_in_center(data, new_centers,p,n)
        new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
        end = time.time()
        #print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    labels_data = [0]*len(data)
    labels_centers = [1]*len(centers)
    labels = labels_data + labels_centers
    X = data + new_centers    
    print_fig(X, labels, "our kmeans")

In [19]:
def real_kmeans(n_clusters, data):
    print("real_kmeans...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(np.array(data).reshape(-1, 1))
    labels_data = [0]*len(data)
    labels_centers = [1]*len(kmeans.cluster_centers_)
    labels = labels_data + labels_centers
    X = data + list(kmeans.cluster_centers_)  
    print_fig(X, labels, "real kmeans")
    return kmeans.cluster_centers_

In [20]:
def encrypted_kmeans(data, centers, n, p, iterations):
    print("encrypted_kmeans...")
    context = get_other_context()
    enc_data = [ts.ckks_tensor(context, [item]) for item in data]
    enc_centers = [ts.ckks_tensor(context, [item]) for item in centers]
    
    start = time.time()
    probs, p_values_sum = prob_to_be_in_center(enc_data, enc_centers,p,n)
    probs, p_values_sum = probs.decrypt().tolist(), p_values_sum.decrypt().tolist()
    
    new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
    end = time.time()

    print("({}/{}) centers = {}, Duration: {} seconds".format(1, iterations, new_centers, end - start))

    for i in range(1, iterations):
        start = time.time()
        enc_data = [ts.ckks_tensor(context, [item]) for item in data]
        enc_centers = [ts.ckks_tensor(context, [item]) for item in centers]
        probs, p_values_sum = prob_to_be_in_center(enc_data, enc_centers,p,n)
        probs, p_values_sum = probs.decrypt().tolist(), p_values_sum.decrypt().tolist()
        new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
        end = time.time()
        print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    labels_data = [0]*len(data)
    labels_centers = [1]*len(centers)
    labels = labels_data + labels_centers
    X = data + new_centers    
    print_fig(X, labels, "encrypted kmeans")

In [21]:
static_data = [random.uniform(0,0.2) for i in range(50)] + [random.uniform(0.5,0.9) for i in range(50)]

In [ ]:
x1 = 0.1
x2 = 0.9
x3 = 0.2
x4 = 0.8
data = [x1, x2, x3, x4]
c1 = 0.48
c2 = 0.49
initial_centers = [c1, c2]

n=4
#p must be even
p=2
iterations = 9

#data = static_data

our_kmeans(data, initial_centers, n, p, iterations)
real_kmeans(len(initial_centers), data)
encrypted_kmeans(data, initial_centers, n, p, iterations)

our_kmeans...
(9/9) centers = [0.15374700627513102, 0.8462529937502712], Duration: 0.0 seconds
real_kmeans...
encrypted_kmeans...
sum of coeff_mod_bit_sizes= 670


C:\Users\dolev\AppData\Local\Temp\ipykernel_24932\455471139.py:122: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
c:\users\dolev\appdata\local\programs\python\python38\lib\site-packages\numpy\ma\core.py:2831: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  _data = np.array(data, dtype=dtype, copy=copy,
C:\Users\dolev\AppData\Local\Temp\ipykernel_24932\455471139.py:122: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


In [ ]:
print("Start Running...")
x1 = 0.1
x2 = 0.9
x3 = 0.2
x4 = 0.8
c = 0.7
print("Creating Context...")
context = get_other_context()
encrypted_x1 = ts.ckks_tensor(context, [x1])
encrypted_x2 = ts.ckks_tensor(context, [x2])
encrypted_x3 = ts.ckks_tensor(context, [x3])
encrypted_x4 = ts.ckks_tensor(context, [x4])

encrypted_c = ts.ckks_tensor(context, [c])

print("Calculating new center...")
start = time.time()
enc_new_center = prob_to_be_in_center(encrypted_x1, encrypted_x2,encrypted_x3,encrypted_x4, encrypted_c)
new_center_dec = enc_new_center.decrypt().tolist()[0]*0.25
end = time.time()

iterations = 4
print("({}/{}) center = {}, Duration: {} seconds".format(1, iterations, new_center_dec, end - start))

for i in range(1, iterations):
    start = time.time()
    new_center_plain = new_center_dec
    encrypted_c = ts.ckks_tensor(context, [new_center_plain])
    enc_new_center = prob_to_be_in_center(encrypted_x1, encrypted_x2,encrypted_x3,encrypted_x4, encrypted_c)
    new_center_dec = enc_new_center.decrypt().tolist()[0]*0.25
    end = time.time()
    print("({}/{}) center = {}, Duration: {} seconds".format(i+1, iterations, new_center_dec, end - start))
